In [1]:
import os
import glob
import re
from math import floor
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import keras_video.utils
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import (ImageDataGenerator, img_to_array)
from keras.layers import Conv2D, BatchNormalization, MaxPool2D, GlobalMaxPool2D, Dense, Dropout, Flatten

In [2]:
def build_mobile(shape=(224, 224, 3)):
    model = keras.applications.mobilenet.MobileNet(
        include_top=False,
        input_shape=shape,
        weights='imagenet')

    trainable = 4
    for layer in model.layers[:-trainable]:
        layer.trainable = False
    for layer in model.layers[-trainable:]:
        layer.trainable = True
    output = GlobalMaxPool2D()
    return keras.Sequential([model, output])

In [3]:
def add_top_model(num_classes):
    mobile = build_mobile()
    # then create our final model
    model = keras.Sequential()
    model.add(mobile)
    # add the convnet with (5, 224, 224, 3) shape
    model.add(Flatten(name="flatten"))
    # here, you can also use GRU or LSTM
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(num_classes, activation="softmax"))
    return model

In [4]:
n_classes = 2
model = add_top_model(n_classes)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 1024)              3228864   
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 256)               262400    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 2)                

In [27]:
train_data_dir = './pigs_tail/train'
validation_data_dir = './pigs_tail/validation'

# Let's use some data augmentaiton
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1. / 255)

# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 16

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    save_to_dir='crop-tail',
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical')

Found 722 images belonging to 2 classes.
Found 52 images belonging to 2 classes.


In [6]:
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

In [ ]:
from sklearn.metrics import classification_report

EPOCHS = 100
# create a "pig_tails_checkpoint" directory before to run that
# because ModelCheckpoint will write models inside
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        'pig_tails_checkpoint/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
]

model.fit(
    train_generator,
    validation_data=validation_generator,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)


Epoch 1/100
46/46 [==============================] - ETA: 0s - loss: 0.1076 - acc: 0.9681
Epoch 00001: saving model to pig_tails_checkpoint\weights.01-0.18.hdf5
46/46 [==============================] - 20s 442ms/step - loss: 0.1076 - acc: 0.9681 - val_loss: 0.1788 - val_acc: 0.9231 - lr: 1.0000e-10
Epoch 2/100
46/46 [==============================] - ETA: 0s - loss: 0.1049 - acc: 0.9681
Epoch 00002: saving model to pig_tails_checkpoint\weights.02-0.18.hdf5
46/46 [==============================] - 26s 559ms/step - loss: 0.1049 - acc: 0.9681 - val_loss: 0.1767 - val_acc: 0.9231 - lr: 1.0000e-10
Epoch 3/100
46/46 [==============================] - ETA: 0s - loss: 0.0877 - acc: 0.9695
Epoch 00003: saving model to pig_tails_checkpoint\weights.03-0.18.hdf5
46/46 [==============================] - 25s 543ms/step - loss: 0.0877 - acc: 0.9695 - val_loss: 0.1769 - val_acc: 0.9231 - lr: 1.0000e-10
Epoch 4/100
46/46 [==============================] - ETA: 0s - loss: 0.0765 - acc: 0.9723
Epoch 0000

In [8]:
def discover_classes():
    # use sub directories names as classes
    classes = [i.split(os.path.sep)[1] for i in glob.glob('./pigs_tail/train/*')]
    classes.sort()

    return classes

In [9]:
def predict(model_, classes, img_path):
    input_im = cv2.imread(img_path)

    input_im = cv2.resize(input_im, (224, 224), interpolation=cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1, 224, 224, 3)
    predictions = model_.predict(input_im)

    closest_guess = None
    index_guess = 0
    prob_guess = 0

    for pre in predictions[0]:
        if pre > prob_guess:
            prob_guess = pre
            closest_guess = index_guess
        index_guess = index_guess + 1

    return classes[closest_guess]

In [10]:
def display_test_result(name, pred, img_path):
    input_im = cv2.imread(img_path)
    input_original = input_im.copy()
    BLACK = [0, 0, 0]
    expanded_image = cv2.copyMakeBorder(input_original, 80, 0, 0, 100, cv2.BORDER_CONSTANT, value=BLACK)
    cv2.putText(expanded_image, pred, (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow(name, expanded_image)

In [11]:
from os import listdir
from os.path import isfile, join

files = [f for f in listdir('./test-img') if isfile(join('./test-img', f))]

In [12]:
cls = discover_classes()
tail_classifier = keras.models.load_model('tail-classifier.hdf5')

for file in files:
    guess = predict(tail_classifier, cls, f'test-img/{file}')
    display_test_result(file, guess, f'test-img/{file}')
    cv2.waitKey(0)

cv2.destroyAllWindows()

In [13]:
#get back the webcam
cv2.destroyAllWindows()